In [1]:
import kimimaro
import numpy as np
import imageio
from skimage import measure
import os
import edt
from skimage import morphology

In [2]:
rootFolder = '/work/boyu/EM_astrocyte/materials_for_paper_VSOT/VSOT_data_paired_with_codes/data/performance_testing_level_2_segmentation/large_dataset_3_annotator_L2345_w_stubby/gt_final_800/stubby_samples/tif_file'
outFolder = '/work/boyu/EM_astrocyte/materials_for_paper_VSOT/VSOT_data_paired_with_codes/data/performance_testing_level_2_segmentation/large_dataset_3_annotator_L2345_w_stubby/gt_final_800/stubby_samples/spine_skel'
if not os.path.exists(outFolder):
    os.makedirs(outFolder)

for filename in os.listdir(rootFolder):
    if filename.endswith('.tif'):
        inputfilename = os.path.splitext(filename)[0]
        vol = imageio.mimread(os.path.join(rootFolder, filename), memtest=False)
        vol = np.stack(vol, 2)
        vol = vol == 2
        vol.astype(int)
        labels = vol
        skels = kimimaro.skeletonize(
            labels,
            teasar_params={
                'scale': 4,
                'const': 500, # physical units
                'pdrf_exponent': 4,
                'pdrf_scale': 100000,
                'soma_detection_threshold': 1100, # physical units
                'soma_acceptance_threshold': 3500, # physical units
                'soma_invalidation_scale': 1.0,
                'soma_invalidation_const': 300, # physical units
                'max_paths': 50, # default None
            },
            dust_threshold=500, # skip connected components with fewer than this many voxels
            anisotropy=(16,16,40), # default True
            fix_branching=True, # default True
            fix_borders=False, # default True
            fill_holes=False, # default False
            fix_avocados=False, # default False
            progress=True, # default False, show progress bar
            parallel=10, # <= 0 all cpu, 1 single process, 2+ multiprocess
            parallel_chunk_size=10, # how many skeletons to process before updating progress bar
        )
        binimg = np.zeros(labels.shape, dtype=np.uint8)
        aa = skels[1].vertices
        verts = skels[1].vertices // (16,16,40)
        verts = np.array(verts)
        verts = verts.astype(int)
        for a in verts:
            binimg[a[0], a[1], a[2]] = 1

        w = imageio.get_writer(os.path.join(outFolder, inputfilename + "_skel.tif"),
                        mode='v')
        lenx, leny, lenz = binimg.shape
        for i in range(lenz):
            w.append_data(binimg[:, :, i])
        w.close()


Skeletonizing Labels: 100%|██████████| 1/1 [00:00<00:00, 37.77it/s]
